# U-Net for Image Segmentation

In [1]:
import cv2
import os
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from skimage.transform import rescale, resize
from sklearn.model_selection import train_test_split
import tensorflow as tf

c:\users\huzai\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def disp(I):
    I = np.uint8(I)
    cv2.imshow('image', I)
    cv2.waitKey(0)

## Function to resize the dataset 
* The aspect ratio of images is preserved while resizing since the shape could be a potential feature of the segmented image.
* The scaling factors in x and y directions are computed for each image keeping ** 384 x 384 ** as the target spatial dimensions.
* The minimum of the scaling factors in the x and y directions is chosen as the final scaling factor.
* By doing so at least one of the dimensions of the resized image will span the corresponding dimension of the CNN input size. 

In [3]:
def resizer(input_folder_path, output_folder_path, CNN_input_size):
    for file_name in os.listdir(input_folder_path):
        img = cv2.imread(os.path.join(input_folder_path, file_name))
        height, width = img.shape[:2]
        scale_x = CNN_input_size[1] / width
        scale_y = CNN_input_size[0] / height
        scale_both = min(scale_x, scale_y)
        new_width = int(scale_both * width)
        new_height = int(scale_both * height)
        img = cv2.resize(img, (new_width, new_height))
        padded_img = np.zeros(CNN_input_size)
        row_low = int((CNN_input_size[0] - new_height) / 2)
        row_high = row_low + new_height - 1
        col_low = int((CNN_input_size[1] - new_width) / 2)
        col_high = col_low + new_width - 1
        padded_img[row_low : row_high + 1, col_low : col_high + 1, :] = img
        cv2.imwrite(os.path.join(output_folder_path, file_name), padded_img)

## Resize Melonoma images

In [3]:
melonoma_input_folder = '../Dataset/Images/melanoma'
melonoma_output_folder = '../Dataset/Images/melonoma-resized'

In [4]:
# resizer(melonoma_input_folder, melonoma_output_folder, CNN_input_size)

## Resize Other images

In [5]:
others_input_folder = '../Dataset/Images/others'
others_output_folder = '../Dataset/Images/others-resized'

In [6]:
# resizer(others_input_folder, others_output_folder, CNN_input_size)

## Resize Ground-Truth images

In [7]:
ground_truth_input_folder = '../Dataset/Images/gt'
ground_truth_output_folder = '../Dataset/Images/gt-resized'

In [8]:
# resizer(ground_truth_input_folder, ground_truth_output_folder, CNN_input_size)

## Read data

In [9]:
X_train = []
Y_train = []

In [10]:
for file_name in os.listdir(melonoma_output_folder):
    train_img = cv2.imread(os.path.join(melonoma_output_folder, file_name))
    X_train.append(train_img)
    train_label = cv2.imread(os.path.join(ground_truth_output_folder, file_name[:12] + '_segmentation.png'), 0)
    Y_train.append(train_label)
for file_name in os.listdir(others_output_folder):
    train_img = cv2.imread(os.path.join(others_output_folder, file_name))
    X_train.append(train_img)
    train_label = cv2.imread(os.path.join(ground_truth_output_folder, file_name[:12] + '_segmentation.png'), 0)
    Y_train.append(train_label)

In [11]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
# Binary Thresholding to undo the blurring caused by cv2.resize()
Y_train[Y_train >= 127] = 255
Y_train[Y_train < 127] = 0
Y_train = np.expand_dims(Y_train, axis = 3) / 255
print(X_train.shape)
print(Y_train.shape)

(2000, 384, 384, 3)
(2000, 384, 384, 1)


In [13]:
np.savez_compressed('../Dataset/Train_Data/train_segmentation', a = X_train, b = Y_train)

In [14]:
data = np.load('../Dataset/Train_Data/train_segmentation.npz')
X = data['a']
Y = data['b']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10, random_state = 7)
print(X.shape)
print(Y.shape)
'''
Expected Output
(2000, 384, 384, 3)
(2000, 384, 384, 1)
'''

(2000, 384, 384, 3)
(2000, 384, 384, 1)


'\nExpected Output\n(2000, 384, 384, 3)\n(2000, 384, 384, 1)\n'

## CNN starts here

In [15]:
CNN_input_size = [384, 384, 3]
trained_segmenter_path = '../Trained/segmenter.h5'

## Mean IoU Metric

In [16]:
def mean_IoU(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

## IoU Loss

In [20]:
def IoU_loss(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return 1 - intersection / union

## Combined IoU Loss with Binary Cross Entropy

In [21]:
def IoU_loss_with_binary_crossentropy(y_true, y_pred):
    return IoU_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)

## U-Net Architecture

In [22]:
inputs = Input(CNN_input_size) 
normalize = Lambda(lambda x : x / 255) (inputs)

conv_1 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (normalize)
conv_1 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (conv_1)
pool_1 = MaxPooling2D((2, 2)) (conv_1)

conv_2 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (pool_1)
conv_2 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (conv_2)
pool_2 = MaxPooling2D((2, 2)) (conv_2)

conv_3 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (pool_2)
conv_3 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (conv_3)
pool_3 = MaxPooling2D((2, 2)) (conv_3)

conv_4 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (pool_3)
conv_4 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (conv_4)
pool_4 = MaxPooling2D((2, 2)) (conv_4)

conv_5 = Conv2D(128, (3, 3), activation = 'elu', padding = 'same') (pool_4)
conv_5 = Conv2D(128, (3, 3), activation = 'elu', padding = 'same') (conv_5)

deconv_1 = Conv2DTranspose(64, (2, 2), activation = 'elu', strides = (2, 2), padding = 'same') (conv_5)
deconv_1 = concatenate([deconv_1, conv_4])

conv_6 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (deconv_1)
conv_6 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (conv_6)

deconv_2 = Conv2DTranspose(32, (2, 2), activation = 'elu', strides = (2, 2), padding = 'same') (conv_6)
deconv_2 = concatenate([deconv_2, conv_3])

conv_7 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (deconv_2)
conv_7 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (conv_7)

deconv_3 = Conv2DTranspose(16, (2, 2), activation = 'elu', strides = (2, 2), padding = 'same') (conv_7)
deconv_3 = concatenate([deconv_3, conv_2])

conv_8 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (deconv_3)
conv_8 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (conv_8)

deconv_4 = Conv2DTranspose(8, (2, 2), activation = 'elu', strides = (2, 2), padding = 'same') (conv_8)
deconv_4 = concatenate([deconv_4, conv_1])

conv_9 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (deconv_4)
conv_9 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (conv_9)

outputs = Conv2D(1, (1, 1), activation = 'sigmoid') (conv_9)
model = Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = dice_loss_with_binary_crossentropy, metrics = [mean_IoU, 'accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 384, 384, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 384, 384, 8)  224         lambda_2[0][0]                   
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 384, 384, 8)  584         conv2d_20[0][0]                  
__________________________________________________________________________________________________
max_poolin

## Training the model

In [23]:
callbacks = [ModelCheckpoint(trained_segmenter_path, monitor = 'val_loss', save_best_only = True, verbose = 2)]

In [24]:
model.fit(X_train, Y_train, validation_data = (X_test, Y_test),
          batch_size = 4, verbose = 1, epochs = 25, callbacks = callbacks, shuffle = True)

Train on 1800 samples, validate on 200 samples
Epoch 1/25
1800/1800 [==============================] - 158s 88ms/step - loss: 1.0185 - mean_IoU: 0.5545 - acc: 0.8778 - val_loss: 0.9906 - val_mean_IoU: 0.5978 - val_acc: 0.9030

Epoch 00001: val_loss improved from inf to 0.99060, saving model to ../Trained/segmenter.h5
Epoch 2/25
1800/1800 [==============================] - 156s 87ms/step - loss: 0.9066 - mean_IoU: 0.6056 - acc: 0.8926 - val_loss: 0.8728 - val_mean_IoU: 0.6173 - val_acc: 0.9140

Epoch 00002: val_loss improved from 0.99060 to 0.87279, saving model to ../Trained/segmenter.h5
Epoch 3/25
1800/1800 [==============================] - 156s 87ms/step - loss: 0.8318 - mean_IoU: 0.6243 - acc: 0.9023 - val_loss: 0.7881 - val_mean_IoU: 0.6334 - val_acc: 0.9187

Epoch 00003: val_loss improved from 0.87279 to 0.78809, saving model to ../Trained/segmenter.h5
Epoch 4/25
1800/1800 [==============================] - 157s 87ms/step - loss: 0.7628 - mean_IoU: 0.6423 - acc: 0.9166 - val_loss

## Mean_IoU = 0.807  
## Accuracy = 0.965